In [4]:
import os
import time

from langchain_core.runnables import RunnablePassthrough
from pydantic import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
from langchain_neo4j import Neo4jGraph
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_core.documents import Document
from langchain_ollama import OllamaEmbeddings
from langchain_experimental.llms.ollama_functions import OllamaFunctions


In [5]:
from neo4j import GraphDatabase
url = "bolt://localhost:7687"
username = "neo4j"
password= "neo4j"

graph = Neo4jGraph(url =url, username=username, password=password)
driver = GraphDatabase.driver(url, auth=(username, password))
session = driver.session()

In [6]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_ollama.llms import OllamaLLM

llm = OllamaLLM(model = "llama3.1:8b-instruct-fp16") 

llm_transformer = LLMGraphTransformer(llm=llm)

In [9]:
from langchain.prompts import PromptTemplate
from langchain_core.prompts import PromptTemplate
from langchain_neo4j import GraphCypherQAChain
CYPHER_GENERATION_TEMPLATE = """You are an AI system that generates Cypher queries to retrieve data from a graph database. 
The graph schema consists of nodes and relationships related to an ontology on:
- Adversarial Autoencoders (AAEs)
- Generative Adversarial Networks (GANs)
- Simple Classification
You will receive a question related to the domain. Based on that, generate a Cypher query that answers the question.

Here is the schema:
{schema}

Question: {question}
"""
CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], template=CYPHER_GENERATION_TEMPLATE
)

CYPHER_QA_TEMPLATE = """You are an assistant that helps to provide human-readable answers to questions based on data retrieved from a graph.
The data in the graph is related to an ontology on:
- Adversarial Autoencoders (AAEs)
- Generative Adversarial Networks (GANs)
- Simple Classification

You will receive the context (results of a Cypher query) and a question. Based on that, return an understandable answer.

Context: {context}
Question: {question}
"""

CYPHER_QA_PROMPT = PromptTemplate(
    input_variables=["context", "question"], template=CYPHER_QA_TEMPLATE
)

graphCypher_chain = GraphCypherQAChain.from_llm(
    OllamaLLM(model = "llama3.1:8b-instruct-fp16", temperature=0.0), 
    graph=graph, 
    cypher_prompt=CYPHER_GENERATION_PROMPT, 
    qa_prompt=CYPHER_QA_PROMPT,
    verbose=True,
    allow_dangerous_requests = True
)

make prompt to describe annetto

In [12]:
graphCypher_chain.invoke({"query": "What are the types of networks used in GANs?"})



> Entering new GraphCypherQAChain chain...
Generated Cypher:
Given the provided graph data, there is no direct information about the specific types of training processes used for Generative Adversarial Networks (GANs). However, we can infer some general information about training processes from the entities and relationships present in the graph.

1. **Training Sessions**: The presence of `ns0__TrainingSession` indicates that sessions are a part of the training process. This could imply structured or organized approaches to training models.

2. **Training Steps**: Entities like `ns0__TrainingSingle`, `ns0__TrainingLoop`, and `ns0__TrainingSingleForwardOnly` suggest different types of steps involved in the training process. These might represent various strategies or methodologies for updating model parameters during training.

3. **Layers and Networks**: The involvement of layers (`ns0__FullyConnectedLayer`, `ns0__ConcatLayer`) and networks (`ns0__Network`) suggests that the architec

CypherSyntaxError: {code: Neo.ClientError.Statement.SyntaxError} {message: Invalid input 'Given': expected
  "ALTER"
  "CALL"
  "CREATE"
  "DEALLOCATE"
  "DELETE"
  "DENY"
  "DETACH"
  "DROP"
  "DRYRUN"
  "ENABLE"
  "FINISH"
  "FOREACH"
  "GRANT"
  "INSERT"
  "LOAD"
  "MATCH"
  "MERGE"
  "NODETACH"
  "OPTIONAL"
  "REALLOCATE"
  "REMOVE"
  "RENAME"
  "RETURN"
  "REVOKE"
  "SET"
  "SHOW"
  "START"
  "STOP"
  "TERMINATE"
  "UNWIND"
  "USE"
  "USING"
  "WITH" (line 1, column 1 (offset: 0))
"Given the provided graph data, there is no direct information about the specific types of training processes used for Generative Adversarial Networks (GANs). However, we can infer some general information about training processes from the entities and relationships present in the graph."
 ^}

In [13]:
from yfiles_jupyter_graphs import GraphWidget
def showGraph():
    session = driver.session()
    widget = GraphWidget(graph = session.run("MATCH (n)-[r]->(m) RETURN n, r, m").graph())
    widget.node_label_mapping = 'label'
    widget.hierarchic_layout()
    widget.set_sidebar(start_with='Data')
    return widget

showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))